In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data-files/movies/metadata_clean.csv')
df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995
1,Jumanji,"['adventure', 'fantasy', 'family']",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"['romance', 'comedy']",101.0,6.5,92.0,1995
3,Waiting to Exhale,"['comedy', 'drama', 'romance']",127.0,6.1,34.0,1995
4,Father of the Bride Part II,['comedy'],106.0,5.7,173.0,1995


In [3]:
orig_df = pd.read_csv('data-files/movies/movies_metadata.csv')
orig_df['overview']

c:\programdata\miniconda3\envs\hoseo\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0        Led by Woody, Andy's toys live happily in his ...
1        When siblings Judy and Peter discover an encha...
2        A family wedding reignites the ancient feud be...
3        Cheated on, mistreated and stepped on, the wom...
4        Just when George Banks has recovered from his ...
                               ...                        
45461          Rising and falling between a man and woman.
45462    An artist struggles to finish his work while a...
45463    When one of her hits goes wrong, a professiona...
45464    In a small town live two brothers, one a minis...
45465    50 years after decriminalisation of homosexual...
Name: overview, Length: 45466, dtype: object

In [4]:
df['overview'] = orig_df['overview']
df['id'] = orig_df['id']
df['overview'] = df['overview'].fillna('')

In [5]:
df.head()

,title,genres,runtime,vote_average,vote_count,year,overview,id
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862
1,Jumanji,"['adventure', 'fantasy', 'family']",104.0,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...,8844
2,Grumpier Old Men,"['romance', 'comedy']",101.0,6.5,92.0,1995,A family wedding reignites the ancient feud be...,15602
3,Waiting to Exhale,"['comedy', 'drama', 'romance']",127.0,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom...",31357
4,Father of the Bride Part II,['comedy'],106.0,5.7,173.0,1995,Just when George Banks has recovered from his ...,11862


In [6]:
# 문장에서 단어 추출 Bow모델 (단어 몇번 나오는지 (빈도수)알수 있음. 단. 문맥을 알수 없음.) / 단어 빈도수가 많을수록 그 단어가 이 문장과 관련 있다는 의미.
# 한 문장의 단어빈도수/전체 문서 단어빈도수 = TFIDF

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
tfidf = TfidfVectorizer(stop_words='english') #stop_words : a, the, he, she....를 제거 하겠다는 의미

tfidf_matrix = tfidf.fit_transform(df['overview'])

In [9]:
tfidf_matrix2=tfidf_matrix.astype('float32')
print(tfidf_matrix.dtype, tfidf_matrix2.dtype)

float64 float32


In [10]:
# 영화 유사도
# cosin, nnf, corr

from sklearn.metrics.pairwise import linear_kernel   #cosin 유사도 계산

In [11]:
cosin_sim = linear_kernel(tfidf_matrix2, tfidf_matrix2)

In [12]:
cosin_sim.shape

(45466, 45466)

In [13]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates()    # drop_duplicates : 중복 제거
indices.head()

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64

In [14]:
idx = indices['The Lion King']
print(idx)

359


In [15]:
cosin_sim.shape
np.max(cosin_sim[idx])
sim_scores = list(enumerate(cosin_sim[idx]))   #enumerate : 순서 번호 입력

sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True) #1번째 기준으로 정렬  

In [16]:
top_sim_scores = sim_scores[1:11]    #top 10개의 데이터 추출
top_sim_scores 

[(34682, 0.24321544),
 (9353, 0.20273764),
 (9115, 0.19010702),
 (42829, 0.17756268),
 (25654, 0.17732804),
 (17041, 0.17427555),
 (27933, 0.16017401),
 (6094, 0.15085855),
 (37409, 0.14859416),
 (3203, 0.1421392)]

In [17]:
movie_indices = [ sim_score[0] for sim_score in top_sim_scores ]
movie_indices

[34682, 9353, 9115, 42829, 25654, 17041, 27933, 6094, 37409, 3203]

In [18]:
df['title'].iloc[movie_indices]

34682    How the Lion Cub and the Turtle Sang a Song
9353                                The Lion King 1½
9115                  The Lion King 2: Simba's Pride
42829                                           Prey
25654                                 Fearless Fagan
17041                                   African Cats
27933              Massaï, les guerriers de la pluie
6094                                       Born Free
37409                                     Sour Grape
3203                                The Waiting Game
Name: title, dtype: object

In [19]:
########

In [20]:
credits_df=pd.read_csv('data-files/movies/credits.csv')
keywords_df=pd.read_csv('data-files/movies/keywords.csv')

In [21]:
credits_df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [22]:
keywords_df.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [23]:
def clean_id(x):
    try:
        return int(x)
    except:
        return np.nan

In [24]:
# df['id'].dtype
# df['id'].apply(lambda id: int(id))   #입력 오류 데이터 있음. 
df['id'] = df['id'].apply(clean_id)

In [25]:
df = df[df['id'].notnull()]

In [26]:
# df['id'].dtype
df['id'] = df['id'].astype('int')
credits_df['id'] = credits_df['id'].astype('int')
keywords_df['id'] = keywords_df['id'].astype('int')

In [27]:
df=df.merge(credits_df, on='id').merge(keywords_df, on='id')   #left_index, right_index, left_on, right_on ....

In [28]:
df.head()

,title,genres,runtime,vote_average,vote_count,year,overview,id,cast,crew,keywords
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,Jumanji,"['adventure', 'fantasy', 'family']",104.0,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...,8844,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,Grumpier Old Men,"['romance', 'comedy']",101.0,6.5,92.0,1995,A family wedding reignites the ancient feud be...,15602,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,Waiting to Exhale,"['comedy', 'drama', 'romance']",127.0,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom...",31357,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,Father of the Bride Part II,['comedy'],106.0,5.7,173.0,1995,Just when George Banks has recovered from his ...,11862,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [29]:
# Convert the stringified objects into the native python objects
from ast import literal_eval    #문자열 -> 코드로 변형 

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    df[feature] = df[feature].apply(literal_eval)

In [30]:
#Print the first cast member of the first movie in df
df.iloc[0]['crew'][0]

{'credit_id': '52fe4284c3a36847f8024f49',
 'department': 'Directing',
 'gender': 2,
 'id': 7879,
 'job': 'Director',
 'name': 'John Lasseter',
 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}

In [31]:
# Extract the director's name. If director is not listed, return NaN
def get_director(x):
    for crew_member in x:
        if crew_member['job'] == 'Director':
            return crew_member['name']
    return np.nan

In [32]:
#Define the new director feature
df['director'] = df['crew'].apply(get_director)

In [33]:
#Print the directors of the first five movies
df['director'].head(10)

0      John Lasseter
1       Joe Johnston
2      Howard Deutch
3    Forest Whitaker
4      Charles Shyer
5       Michael Mann
6     Sydney Pollack
7       Peter Hewitt
8        Peter Hyams
9    Martin Campbell
Name: director, dtype: object

In [34]:
# 데이터가 리스트라면 이름만 뽑고, 아니라면 [] 표시해라~
# Returns the list top 3 elements or entire list; whichever is more.
def generate_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names
    
    #Return empty list in case of missing/malformed data
    return []

In [35]:
#Apply the generate_list function to cast and keywords
df['cast'] = df['cast'].apply(generate_list)
df['keywords'] = df['keywords'].apply(generate_list)

In [36]:
df['cast'][0] , df['keywords'][0]

(['Tom Hanks', 'Tim Allen', 'Don Rickles'], ['jealousy', 'toy', 'boy'])

In [37]:
#Only consider a maximum of 3 genres
df['genres'] = df['genres'].apply(lambda x: x[:3])          #장르 3개까지만 가져오기~

In [38]:
# Print the new features of the first 5 movies along with title
df[['title', 'cast', 'director', 'keywords', 'genres']].head()

,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[animation, comedy, family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[adventure, fantasy, family]"
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]","[romance, comedy]"
3,Waiting to Exhale,"[Whitney Houston, Angela Bassett, Loretta Devine]",Forest Whitaker,"[based on novel, interracial relationship, sin...","[comedy, drama, romance]"
4,Father of the Bride Part II,"[Steve Martin, Diane Keaton, Martin Short]",Charles Shyer,"[baby, midlife crisis, confidence]",[comedy]


In [39]:
# Function to sanitize data to prevent ambiguity. It removes spaces and converts to lowercase 
# 대소문자, 공백 처리하기 
def sanitize(x):
    if isinstance(x, list):
        #Strip spaces and convert to lowercase
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [40]:
#Apply the generate_list function to cast, keywords, director and genres
for feature in ['cast', 'director', 'genres', 'keywords']:
    df[feature] = df[feature].apply(sanitize)

In [41]:
df.head(1)

,title,genres,runtime,vote_average,vote_count,year,overview,id,cast,crew,keywords,director
0,Toy Story,"[animation, comedy, family]",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862,"[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy]",johnlasseter


In [42]:
#Function that creates a soup out of the desired metadata
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

In [43]:
# Create the new soup feature
df['soup'] = df.apply(create_soup, axis=1)

In [44]:
df['soup'][:2]        #주요 단어 데이터 한문장에 취합

0    jealousy toy boy tomhanks timallen donrickles ...
1    boardgame disappearance basedonchildren'sbook ...
Name: soup, dtype: object

In [49]:
df.to_csv('data-files/movies/soup.csv', index=False)    #인덱스 빼고 저장 (keywords,cast,director,genres 합)

In [48]:
df.index , df.shape

(Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                 9,
             ...
             46618, 46619, 46620, 46621, 46622, 46623, 46624, 46625, 46626,
             46627],
            dtype='int64', length=46628),
 (46628, 13))

In [ ]:
###############

In [1]:
import pandas as pd

In [2]:
df2 = pd.read_csv('data-files/movies/soup.csv')

In [3]:
df2[:1]

,title,genres,runtime,vote_average,vote_count,year,overview,id,cast,crew,keywords,director,soup
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862,"['tomhanks', 'timallen', 'donrickles']","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","['jealousy', 'toy', 'boy']",johnlasseter,jealousy toy boy tomhanks timallen donrickles ...


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english') #stop_words : a, the, he, she....를 제거 하겠다는 의미
tfidf_mat = tfidf.fit_transform(df2['soup'])

In [6]:
tfidf_mat.dtype
tfidf_mat2=tfidf_mat.astype('float32')
print(tfidf_mat.dtype, tfidf_mat2.dtype)

float64 float32


In [7]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_mat2, tfidf_mat2)

In [9]:
cosine_sim.shape

(46628, 46628)

In [16]:
indices = pd.Series(df2.index, index=df2['title']).drop_duplicates()    # drop_duplicates : 중복 제거
indices.head()

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64

In [18]:
idx = indices['The Lion King']
print(idx)

359


In [20]:
import numpy as np
cosine_sim.shape
np.max(cosine_sim[idx])
sim_scores = list(enumerate(cosine_sim[idx]))   #enumerate : 순서 번호 입력

sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True) #1번째 기준으로 정렬  

In [23]:
top_sim_scores = sim_scores[1:11]    #top 10개의 데이터 추출
top_sim_scores 

[(27768, 0.26719174),
 (33119, 0.25987962),
 (29607, 0.22471112),
 (40904, 0.22471112),
 (40913, 0.22471112),
 (270, 0.21747696),
 (986, 0.21430598),
 (29198, 0.20901513),
 (811, 0.2054554),
 (15209, 0.19960591)]

In [24]:
movie_indices = [ sim_score[0] for sim_score in top_sim_scores ]
movie_indices

[27768, 33119, 29607, 40904, 40913, 270, 986, 29198, 811, 15209]

In [25]:
df2['title'].iloc[movie_indices]

27768                                 The Little Matchgirl
33119                                          The Prophet
29607                                          Cheburashka
40904                   VeggieTales: Josh and the Big Wall
40913    VeggieTales: Minnesota Cuke and the Search for...
270                                       Man of the House
986                                               Infinity
29198                                      Superstar Goofy
811                            The Adventures of Pinocchio
15209             Spiderman: The Ultimate Villain Showdown
Name: title, dtype: object